In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Context_Aware_Description_Using_MiniLM_ViT_GPT2/


Mounted at /content/drive
/content/drive/My Drive/Context_Aware_Description_Using_MiniLM_ViT_GPT2


In [2]:
from collections import defaultdict
import pandas as pd
DATASET_PATH = 'Flickr8K'
filepath = f'{DATASET_PATH}/captions.txt'

In [3]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import pandas as pd
import os
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import GPT2Tokenizer, ViTModel, ViTFeatureExtractor



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
import torch
from torch.utils.data import DataLoader
from transformers import get_scheduler
from torch.optim import AdamW


from tqdm import tqdm
import os

# === Model Setup ===
model = GPT2PrefixCaptionModel().cuda()

# === Dataloader ===
dataloader = DataLoader(
    dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0,        # Keep 0 for Colab
    pin_memory=False
)

# === Optimizer & Scheduler ===
optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)
num_epochs = 3
num_training_steps = num_epochs * len(dataloader)

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=100,
    num_training_steps=num_training_steps
)

# === Save Directory ===
save_dir = "./checkpoints"
os.makedirs(save_dir, exist_ok=True)

# === Training Loop ===
model.train()

for epoch in range(num_epochs):
    epoch_loss = 0.0
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for step, batch in enumerate(progress_bar):
        # Move batch to GPU
        batch = {k: v.cuda() if isinstance(v, torch.Tensor) else v for k, v in batch.items()}

        outputs = model(
            image_feat=batch['image_feat'],
            context_embed=batch['context_embed'],
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            labels=batch['input_ids']
        )

        loss = outputs.loss
        epoch_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        progress_bar.set_postfix(loss=loss.item())

    # === Save Checkpoint After Each Epoch ===
    checkpoint_path = os.path.join(save_dir, f"gpt2_caption_epoch{epoch+1}.pt")
    torch.save(model.state_dict(), checkpoint_path)
    print(f"✅ Saved checkpoint to {checkpoint_path}")


Epoch 1/3: 100%|██████████| 1012/1012 [1:52:20<00:00,  6.66s/it, loss=0.58]


✅ Saved checkpoint to ./checkpoints/gpt2_caption_epoch1.pt


Epoch 2/3: 100%|██████████| 1012/1012 [04:06<00:00,  4.10it/s, loss=0.396]


✅ Saved checkpoint to ./checkpoints/gpt2_caption_epoch2.pt


Epoch 3/3: 100%|██████████| 1012/1012 [04:07<00:00,  4.09it/s, loss=0.293]


✅ Saved checkpoint to ./checkpoints/gpt2_caption_epoch3.pt
